In [ ]:
pip install pysus

In [ ]:
pip install -U https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
pip install -U pandas-profiling

In [10]:
from pysus.online_data import SINAN
import pandas
import pandas_profiling as pp
from pandas_profiling import ProfileReport

In [11]:
SINAN.list_diseases()

['Animais Peçonhentos',
 'Botulismo',
 'Chagas',
 'Colera',
 'Coqueluche',
 'Dengue',
 'Difteria',
 'Esquistossomose',
 'Febre Amarela',
 'Febre Maculosa',
 'Febre Tifoide',
 'Hanseniase',
 'Hantavirose',
 'Hepatites Virais',
 'Intoxicação Exógena',
 'Leishmaniose Visceral',
 'Leptospirose',
 'Leishmaniose Tegumentar',
 'Malaria',
 'Meningite',
 'Peste',
 'Poliomielite',
 'Raiva Humana',
 'Tétano Acidental',
 'Tétano Neonatal',
 'Tuberculose',
 'Violência Domestica']

In [12]:
def change_types(df):
  for columns in df.columns:
      if columns.startswith("DT_"):
          df[columns] = df[columns].astype('datetime64[ns]')
      elif columns == 'NU_ANO':
          df[columns] = df[columns].astype(int)
      elif columns == 'RESUL_HIS':
          df[columns] = df[columns].astype(str)
      else:
          try:
              df[columns] = df[columns].astype(int)
          except:
              try:
                  df[columns] = df[columns].astype(float)
              except:
                  df[columns] = df[columns].astype(str)

In [13]:
def diseases_per_year(disease, states):
  years = []
  db_diseasesYear = {}
  state_disease = {}

  for state in states:
    for item in SINAN.get_available_years(state, disease):
      years.append(item[6:8])
    
    for year in years:
      db_diseasesYear[f'{year}'] = SINAN.download(state, int(f'20{year}'), disease)

    state_disease[state] = db_diseasesYear

  return state_disease


In [14]:
disease = 'Malaria'
states = ['SP', 'RJ']

df_sprj_malaria = diseases_per_year(disease, states)


In [15]:
for state in df_sprj_malaria.keys():
  for year in df_sprj_malaria[state].keys():
    change_types(df_sprj_malaria[state][year])

In [16]:
teste = df_sprj_malaria['SP']['17']


In [ ]:
for state in df_sprj_malaria.keys():
  for year in df_sprj_malaria[state].keys():
    profile = ProfileReport(df_sprj_malaria[state][year], title=f"Report: {disease} em {state}/20{year}", explorative=True)
    profile.to_file(f"report{state}{year}.html")